In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import requests
import io
from PIL import Image
from time import sleep
import time
import random
import base64
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.common.exceptions import WebDriverException
import os
import pandas as pd
import numpy as np


In [2]:
#Crawl basic phone data
# Declare browser
s = Service(r"D:\CodeUIT\Data\Crawl-Data-Phone-Shop\chromedriver.exe")

# Khởi tạo trình duyệt Chrome với tùy chọn toàn màn hình
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=s, options=chrome_options)

# Open URL
driver.get("https://www.thegioididong.com/cap-dien-thoai-cap-am-thanh#c=58&p=237385,237386,237387,237388&o=14&pi=7")
sleep(random.randint(5,10))

# Scroll down
for i in range(5):
    scroll_script = "window.scrollBy(0, window.innerHeight);"
    try:
        driver.execute_script(scroll_script)
    except WebDriverException as e:
        print("WebDriverException:", e)
    time.sleep(1) 

# Scroll up
for i in range(20):
    scroll_script = "window.scrollBy(0, -window.innerHeight);"
    try:
        driver.execute_script(scroll_script)
    except WebDriverException as e:
        print("WebDriverException:", e)
    time.sleep(1)
    
# elems2 = driver.find_elements(By.CLASS_NAME , "jBwCF ")
# links = [elem.get_attribute('src') for elem in elems2 if elem.get_attribute('type') == 'product' and elem.get_attribute('src').count("data:image") <=0]

# ================================ GET id
elems = driver.find_elements(By.CSS_SELECTOR, '.item.__cate_58 a')
id_product = [elem.get_attribute('data-id') for elem in elems if elem.get_attribute('data-id') is not None]

# # ================================ GET title
elems = driver.find_elements(By.CSS_SELECTOR, '.item.__cate_58 a')
title = [elem.get_attribute('data-name') for elem in elems if elem.get_attribute('data-name') is not None]

# ================================ GET img_thumb  .item.ajaxed.__cate_42
# elems = driver.find_elements(By.CLASS_NAME, 'thumb')
elems = driver.find_elements(By.CSS_SELECTOR, '.item-img.item-img_58 img')
link_img_thumb = [elem.get_attribute('src') for elem in elems if elem.get_attribute('src') is not None and "Products" in elem.get_attribute('src')]

# ================================ GET href
elems = driver.find_elements(By.CSS_SELECTOR, '.item.__cate_58 a')
href = [elem.get_attribute('href') for i, elem in enumerate(elems) if i % 3 == 0]

# ================================ GET price
elems = driver.find_elements(By.CSS_SELECTOR, '.item.__cate_58 a')
price = [elem.get_attribute('data-price') for elem in elems if elem.get_attribute('data-price') is not None]

# Clean price
price = [int(float(number)) for number in price]

# ================================ GET price pre discount
pre_discount_list, discount_idx, discount_percent_list = [], [], []
for i in range(1, len(title)+1):
    try:
        discount_percent = driver.find_element("xpath", "/html/body/div[6]/section/div[3]/ul/li[{}]/a[1]/div[3]/span".format(i))
        discount_percent_list.append(discount_percent.text)

        pre_discount = driver.find_element("xpath", "/html/body/div[6]/section/div[3]/ul/li[{}]/a[1]/div[3]/p".format(i))
        pre_discount_list.append(pre_discount.text)

        
        print(i)
        discount_idx.append(i)
    except NoSuchElementException:
        print("No Such Element Exception " + str(i))


# ================================ GET brand
elems = driver.find_elements(By.CSS_SELECTOR, '.item.__cate_58 a')
brand = [elem.get_attribute('data-brand') for elem in elems if elem.get_attribute('data-brand') is not None]


# Close browser
driver.close() 

No Such Element Exception 1
No Such Element Exception 2
No Such Element Exception 3
No Such Element Exception 4
5
No Such Element Exception 6
No Such Element Exception 7
No Such Element Exception 8
9
10
11
No Such Element Exception 12
13
No Such Element Exception 14
No Such Element Exception 15
No Such Element Exception 16
No Such Element Exception 17
No Such Element Exception 18
19
20
21
No Such Element Exception 22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
No Such Element Exception 39
40
No Such Element Exception 41
42
43
44
No Such Element Exception 45
46
No Such Element Exception 47
No Such Element Exception 48
49
50
No Such Element Exception 51
No Such Element Exception 52
No Such Element Exception 53
No Such Element Exception 54
No Such Element Exception 55
56
57
58
No Such Element Exception 59
No Such Element Exception 60
No Such Element Exception 61
No Such Element Exception 62
63
64
65
No Such Element Exception 66
67
68
69
No Such Element Exception 70
71
72
73
74
75
76
7

In [3]:
#Clean discount_list
for i, number in enumerate(pre_discount_list):
    if pre_discount_list[i] and pre_discount_list[i] != '':
        if '.' in pre_discount_list[i]:
            pre_discount_list[i] = int(pre_discount_list[i].replace('.', '').replace('₫', ''))

In [4]:
#Merge & create excel
df1 = pd.DataFrame(list(zip(id_product, title, brand ,href ,link_img_thumb , price)), columns = ['id', 'title' ,'brand', 'href', 'link_img_thumb', 'price'])
# Đặt tên tệp Excel và sheet
excel_file = 'cable.xlsx'
sheet_name = 'Sheet1'

df1['index_']= np.arange(1, len(df1) + 1)

df2 = pd.DataFrame(list(zip(pre_discount_list ,discount_percent_list, discount_idx)), columns = ['pre_discount', 'discount_percent','discount_idx'])
df3 = df1.merge(df2, how='left', left_on='index_', right_on='discount_idx')

# Lưu DataFrame vào tệp Excel
df3.to_excel(excel_file, sheet_name=sheet_name, index=False, engine='openpyxl')

In [5]:
len(link_img_thumb)

143

In [6]:
df = pd.read_excel('cable.xlsx')

link_img_thumb = df['link_img_thumb'].to_list()
href = df['href'].to_list()
id_product = df['id'].to_list()

print(href)

['https://www.thegioididong.com/cap-dien-thoai/cap-type-c-lightning-mfi-0-9m-anker-322-a81b5', 'https://www.thegioididong.com/cap-dien-thoai/cap-type-c-lightning-0-2m-xmobile-l2203', 'https://www.thegioididong.com/cap-dien-thoai/cap-type-c-lightning-2m-baseus-explorer-cb000040', 'https://www.thegioididong.com/cap-dien-thoai/cap-type-c-lightning-1m-xmobile-qc-tc06-1000', 'https://www.thegioididong.com/cap-dien-thoai/cap-type-c-type-c-1-25m-mazer-c2c125', 'https://www.thegioididong.com/cap-dien-thoai/cap-type-c-type-c-1m-xmobile-ds101', 'https://www.thegioididong.com/cap-dien-thoai/cap-lightning-ava-dr-l04', 'https://www.thegioididong.com/cap-dien-thoai/cap-lightning-1m-hydrus-cs-c-021-xanh-navy', 'https://www.thegioididong.com/cap-dien-thoai/cap-type-c-1m-baseus-glimmer-cb000024-trang', 'https://www.thegioididong.com/cap-dien-thoai/cap-type-c-1m-baseus-glimmer-cb000024-den', 'https://www.thegioididong.com/cap-dien-thoai/cap-type-c-1m-baseus-explorer-cb000042-trang-xanh', 'https://www.th

In [7]:
def download_image(download_path, url, index):
    try:
        if url.startswith('data:image/'):
            # Trích xuất dữ liệu hình ảnh được mã hóa Base64 từ data URI
            image_data = url.split(',')[1]
            image_bytes = base64.b64decode(image_data)

            # Tạo một hình ảnh PIL từ các byte và lưu nó
            image = Image.open(io.BytesIO(image_bytes))
            # file_path = title[i] + file_name

            with open(file_path, "wb") as f:
                image.save(f, "JPEG")
        else:
            image_content = requests.get(url).content
            image_file = io.BytesIO(image_content)
            image = Image.open(image_file)
            # file_path = title[i] + file_name
            
            file_path = os.path.join(download_path, f"{id_product[i]}.jpg")

            with open(file_path, "wb") as f:
                image.save(f, "JPEG")

        print("Success ", index)
    except Exception as e:
        print('FAILED -', e)


urls = link_img_thumb

dowloadPath = r'D:\CodeUIT\Data\imgs\img_product'
for i, url in enumerate(urls):
	download_image(dowloadPath, url, i)

Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
Success  11
Success  12
Success  13
Success  14
Success  15
Success  16
Success  17
Success  18
Success  19
Success  20
Success  21
Success  22
Success  23
Success  24
Success  25
Success  26
Success  27
Success  28
Success  29
Success  30
Success  31
Success  32
Success  33
Success  34
Success  35
Success  36
Success  37
Success  38
Success  39
Success  40
Success  41
Success  42
Success  43
Success  44
Success  45
Success  46
Success  47
Success  48
Success  49
Success  50
Success  51
Success  52
Success  53
Success  54
Success  55
Success  56
Success  57
Success  58
Success  59
Success  60
Success  61
Success  62
Success  63
Success  64
Success  65
Success  66
Success  67
Success  68
Success  69
Success  70
Success  71
Success  72
Success  73
Success  74
Success  75
Success  76
Success  77
Success  78
Success  79
Success  80
Success  81
Success  82
Success  83
Su

In [15]:
#Crawl parameters data
list_parameters=[]
for href_items in href:
# for i in range(1):
    href_items = href_items + '#top-tskt'

    # Declare browser
    s = Service(r"D:\CodeUIT\Data\Crawl-Data-Phone-Shop\chromedriver.exe")

    # Browser full window
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=s, options=chrome_options)

    
    #Open browser 
    driver.get(href_items)
    sleep(random.randint(1,2))

    # Scroll down
    # for i in range(2):
    #     scroll_script = "window.scrollBy(0, window.innerHeight);"
    #     driver.execute_script(scroll_script)
    #     time.sleep(1) 

    elems = driver.find_elements(By.CSS_SELECTOR, '.parameter-item')
    # id_product = [elem.get_attribute('data-id') for elem in elems if elem.get_attribute('data-id') is not None]
    parameters = [elem.text for elem in elems]

    list_parameters.append(parameters[0])


    #Close browser
    driver.close()

In [5]:
df = pd.read_excel('list_parameters.xlsx')
list_parameters = df['color_item'].to_list()

In [6]:
list_parameters

['Công nghệ/Tiện ích:\nHỗ trợ sạc nhanh\nChức năng:\nSạc Truyền dữ liệu\nĐầu vào:\nUSB Type-C\nĐầu ra:\nLightning: 20 W\nĐộ dài dây:\n0.9 m\nCông suất tối đa:\n20 W\nSản xuất tại:\nTrung Quốc\nThương hiệu của:\nTrung Quốc\nHãng:\nAnker. Xem thông tin hãng',
 'Công nghệ/Tiện ích:\nHỗ trợ sạc nhanh\nChức năng:\nSạc\nĐầu vào:\nUSB Type-C\nĐầu ra:\nLightning: 5V - 2.4A, 9V - 3A\nĐộ dài dây:\n0.2 m\nCông suất tối đa:\n27 W\nSản xuất tại:\nTrung Quốc\nThương hiệu của:\nTrung Quốc\nHãng:\nXmobile. Xem thông tin hãng',
 'Công nghệ/Tiện ích:\nSmart Power - OFF Power Delivery\nChức năng:\nSạc Truyền dữ liệu\nĐầu vào:\nUSB Type-C\nĐầu ra:\nLightning: 20 W\nĐộ dài dây:\n2 m\nCông suất tối đa:\n20 W\nSản xuất tại:\nTrung Quốc\nThương hiệu của:\nTrung Quốc\nHãng:\nBaseus. Xem thông tin hãng',
 'Công nghệ/Tiện ích:\nHỗ trợ sạc nhanh\nChức năng:\nSạc Truyền dữ liệu\nĐầu vào:\nUSB Type-C\nĐầu ra:\nLightning: 5V - 2.4A, 9V - 2.22A (Max 20 W)\nĐộ dài dây:\n1 m\nCông suất tối đa:\n20 W\nSản xuất tại:\nTru

In [38]:
tech = []
function =[]
input_ =[]
output =[]
length = []
maximum = []
madein = []
brandof = [] 
brand = []
#Clean parameters data
for i, data in enumerate(list_parameters):
    data = data.replace(":", "")
    # Tách dữ liệu thành danh sách các dòng
    data = data.replace('\n', ' ')
    # Tìm vị trí của các thông số trong dữ liệu
    if "Xem thông tin" in data:
        index = data.find(". Xem")
        data = data[0:index]
    if "Jack kết nối" in data:
        index_start = data.find("Jack kết nối")
        index_end = data.find("Dòng sạc")
        data = data[0:index_start-1] + data[index_end:]
    
    start_index = {
        "Công nghệ/Tiện ích": data.find("Công nghệ/Tiện ích"),
        "Chức năng": data.find("Chức năng"),
        "Đầu vào": data.find("Đầu vào"),
        "Đầu ra": data.find("Đầu ra"),
        "Độ dài dây": data.find("Độ dài dây"),
        "Công suất tối đa": data.find("Công suất tối đa"),
        "Sản xuất tại": data.find("Sản xuất tại"),
        "Thương hiệu của": data.rfind("Thương hiệu của"),
        "Hãng": data.rfind("Hãng")

    }

    # Tìm vị trí của các thông số kết thúc
    end_index = {
        "Công nghệ/Tiện ích": data.find("Chức năng"),
        "Chức năng": data.find("Đầu vào"),
        "Đầu vào": data.find("Đầu ra"),
        "Đầu ra": data.find("Độ dài dây"),
        "Độ dài dây": data.find("Công suất tối đa"),
        "Công suất tối đa": data.find("Sản xuất tại"),
        "Sản xuất tại": data.find("Thương hiệu của"),
        "Thương hiệu của": data.rfind("Hãng"),
        "Hãng": len(data)
    }

    # Khởi tạo một từ điển trống
    chargingadapter_info = {}

    # Duyệt qua các thông số để lấy giá trị
    for key in start_index.keys():
        if start_index[key] != -1 and end_index[key] != -1:
            value = data[start_index[key] + len(key + " "):end_index[key]]
            chargingadapter_info[key] = value

    if 'Công nghệ/Tiện ích' in chargingadapter_info:
        tech.append(chargingadapter_info['Công nghệ/Tiện ích'])
    else:
        tech.append(None)
    if 'Chức năng' in chargingadapter_info:
        function.append(chargingadapter_info['Chức năng'])
    else:
        function.append(None)
    if 'Đầu vào' in chargingadapter_info:
        input_.append(chargingadapter_info['Đầu vào'])
    else:
        input_.append(None) 
    if 'Đầu ra' in chargingadapter_info:
        output.append(chargingadapter_info['Đầu ra'])
    else:
        output.append(None) 
    if 'Công suất tối đa' in chargingadapter_info:
        maximum.append(chargingadapter_info['Công suất tối đa'])
    else:
        maximum.append(None) 
    if 'Độ dài dây' in chargingadapter_info:
        length.append(chargingadapter_info['Độ dài dây'])
    else:
        length.append(None) 
    if 'Sản xuất tại' in chargingadapter_info:
        madein.append(chargingadapter_info['Sản xuất tại'])
    else:
        madein.append(None)
    if 'Thương hiệu của' in chargingadapter_info:
        brandof.append(chargingadapter_info['Thương hiệu của'])
    else:
        brandof.append(None)
    if 'Hãng' in chargingadapter_info:
        brand.append(chargingadapter_info['Hãng'])
    else:
        brand.append(None)
    print(chargingadapter_info)


{'Công nghệ/Tiện ích': 'Hỗ trợ sạc nhanh ', 'Chức năng': 'Sạc Truyền dữ liệu ', 'Đầu vào': 'USB Type-C ', 'Đầu ra': 'Lightning 20 W ', 'Độ dài dây': '0.9 m ', 'Công suất tối đa': '20 W ', 'Sản xuất tại': 'Trung Quốc ', 'Thương hiệu của': 'Trung Quốc ', 'Hãng': 'Anker'}
{'Công nghệ/Tiện ích': 'Hỗ trợ sạc nhanh ', 'Chức năng': 'Sạc ', 'Đầu vào': 'USB Type-C ', 'Đầu ra': 'Lightning 5V - 2.4A, 9V - 3A ', 'Độ dài dây': '0.2 m ', 'Công suất tối đa': '27 W ', 'Sản xuất tại': 'Trung Quốc ', 'Thương hiệu của': 'Trung Quốc ', 'Hãng': 'Xmobile'}
{'Công nghệ/Tiện ích': 'Smart Power - OFF Power Delivery ', 'Chức năng': 'Sạc Truyền dữ liệu ', 'Đầu vào': 'USB Type-C ', 'Đầu ra': 'Lightning 20 W ', 'Độ dài dây': '2 m ', 'Công suất tối đa': '20 W ', 'Sản xuất tại': 'Trung Quốc ', 'Thương hiệu của': 'Trung Quốc ', 'Hãng': 'Baseus'}
{'Công nghệ/Tiện ích': 'Hỗ trợ sạc nhanh ', 'Chức năng': 'Sạc Truyền dữ liệu ', 'Đầu vào': 'USB Type-C ', 'Đầu ra': 'Lightning 5V - 2.4A, 9V - 2.22A (Max 20 W) ', 'Độ dài dây

In [40]:
new_df2 = pd.DataFrame(list(zip(tech, function, input_ ,output , length, maximum, madein, brandof, brand)), columns = ['tech', 'function', 'input_' ,'output' , 'length', 'maximum', 'madein', 'brandof', 'brand'])
new_df2.to_excel('cable_info.xlsx', sheet_name='Sheet 1', index=False, engine='openpyxl')

In [39]:
for i, item in enumerate(tech):
    if item is None or item == '':
        tech[i] = 'Không có'

In [ ]:
brand

In [47]:
def download_image_details(download_path, url, index):
    try:
        if url.startswith('data:image/'):
            # Trích xuất dữ liệu hình ảnh được mã hóa Base64 từ data URI
            image_data = url.split(',')[1]
            image_bytes = base64.b64decode(image_data)

            # Tạo một hình ảnh PIL từ các byte và lưu nó
            image = Image.open(io.BytesIO(image_bytes))
            # file_path = title[i] + file_name

            with open(file_path, "wb") as f:
                image.save(f, "JPEG")
        else:
            image_content = requests.get(url).content
            image_file = io.BytesIO(image_content)
            image = Image.open(image_file)
            # file_path = title[i] + file_name
            
            file_path = os.path.join(download_path, f"{index}.jpg")

            with open(file_path, "wb") as f:
                image.save(f, "JPEG")

        print("Success ", index)
    except Exception as e:
        print('FAILED -', e)


# urls = link_img_thumb

# parent_folder = r'D:\CodeUIT\Data\imgs'
# folder_name = 'abc'

# folder_path = os.path.join(parent_folder, folder_name)

# os.makedirs(folder_path, exist_ok=True)

# dowloadPath = r'D:\CodeUIT\Data\imgs\img_product'
# for i, url in enumerate(urls):
# 	download_image(dowloadPath, url, i)

In [41]:
df = pd.read_excel('cable.xlsx')
href = df['href'].to_list()
id_product = df['id'].to_list()

In [42]:
href

['https://www.thegioididong.com/cap-dien-thoai/cap-type-c-lightning-mfi-0-9m-anker-322-a81b5',
 'https://www.thegioididong.com/cap-dien-thoai/cap-type-c-lightning-0-2m-xmobile-l2203',
 'https://www.thegioididong.com/cap-dien-thoai/cap-type-c-lightning-2m-baseus-explorer-cb000040',
 'https://www.thegioididong.com/cap-dien-thoai/cap-type-c-lightning-1m-xmobile-qc-tc06-1000',
 'https://www.thegioididong.com/cap-dien-thoai/cap-type-c-type-c-1-25m-mazer-c2c125',
 'https://www.thegioididong.com/cap-dien-thoai/cap-type-c-type-c-1m-xmobile-ds101',
 'https://www.thegioididong.com/cap-dien-thoai/cap-lightning-ava-dr-l04',
 'https://www.thegioididong.com/cap-dien-thoai/cap-lightning-1m-hydrus-cs-c-021-xanh-navy',
 'https://www.thegioididong.com/cap-dien-thoai/cap-type-c-1m-baseus-glimmer-cb000024-trang',
 'https://www.thegioididong.com/cap-dien-thoai/cap-type-c-1m-baseus-glimmer-cb000024-den',
 'https://www.thegioididong.com/cap-dien-thoai/cap-type-c-1m-baseus-explorer-cb000042-trang-xanh',
 'htt

In [115]:
href_test = []
for i, data in enumerate(href):
    href_test.append(data)
    if i==5: break

In [44]:
len(id_product)

143

In [121]:
id_failed = []
id_success = []
for i,href_items in enumerate(href):
# for i in range(1):
    s = Service(r"D:\CodeUIT\Data\Crawl-Data-Phone-Shop\chromedriver.exe")

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=s, options=chrome_options)

    # driver.get("https://www.thegioididong.com/dtdd-apple-iphone")
    # driver.get("https://www.thegioididong.com/dtdd/samsung-galaxy-s23-fe")
    # driver.get("https://www.thegioididong.com/dtdd-samsung")
    driver.get(href_items)
    # sleep(random.randint(1,2))
    sleep(0.5)
    # Scroll down
    for scroll_i in range(2):
        scroll_script = "window.scrollBy(0, window.innerHeight);"
        driver.execute_script(scroll_script)
        time.sleep(1) 

    # # Scroll up
    # for i in range(5):
    #     scroll_script = "window.scrollBy(0, -window.innerHeight);"
    #     driver.execute_script(scroll_script)
    #     time.sleep(1)


    # elems = driver.find_elements(By.CSS_SELECTOR, '.parameter-item')
    # parameters_details = [elem.text for elem in elems]

    elems = driver.find_elements(By.CSS_SELECTOR, '.owl-stage .owl-item.active a img')
    pics_act = [elem.get_attribute('src') for elem in elems ]

    elems = driver.find_elements(By.CSS_SELECTOR, '.owl-stage .owl-item a img' )
    pics = [elem.get_attribute('data-src') for elem in elems  ]

    pics_act_clean = []
    pics_clean = []

    for data in pics_act:
        if data and isinstance(data, str) and 'Slider' in data:
            pics_act_clean.append(data)
    
    for data in pics:
        if data and isinstance(data, str) and 'Slider' in data:
            pics_clean.append(data)

    # print(pics_act_clean)
    # print(pics_clean)

    pics_act = pics_act_clean
    pics = pics_clean

    if pics_act and pics:
        pics.insert(0, pics_act[0])
    
        urls = pics

        parent_folder = r'D:\CodeUIT\Data\imgs\imgs_details'
        folder_name = str(id_product[i])

        folder_path = os.path.join(parent_folder, folder_name)

        os.makedirs(folder_path, exist_ok=True)

        dowloadPath = folder_path

        for z, url in enumerate(urls):
            download_image_details(dowloadPath, url, z)

        id_success.append(id_product[i])
        print('------Successful ', i)
    else:
        id_failed.append(id_product[i])


   
    driver.close()

Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
------Successful  0
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
------Successful  1
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
------Successful  2
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
------Successful  3
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
------Successful  4
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
------Successful  5
FAILED - cannot write mode RGBA as JPEG
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
FAILED - cannot write mode RGBA as JPEG
Success  8
------Successfu

In [123]:
id_failed

[313194,
 302953,
 309835,
 314975,
 306994,
 317163,
 317164,
 317981,
 312966,
 275953,
 309864,
 314203,
 313202,
 313199,
 316771,
 317221]

In [ ]:
i=0
for data in pics_clean:
    print(data, '\n')
    i+=1
    

print(i)

In [74]:
id_success

[]

In [45]:
i=0
for data in pics_act:
    if data is not None and '305659' in data and 'thumb' not in data:
        print(data, '\n')
        i+=1
print(i)
# for data in pics_act:
#     if data is not None :
#         print(data, '\n')
print(pics_act)

0
[]


In [45]:
df = pd.read_excel('cable.xlsx')
href = df['href'].to_list()
id_product = df['id'].to_list()
print(id_product)

[312263, 311309, 310592, 309714, 308532, 307838, 240626, 316872, 316457, 316456, 316454, 315202, 312266, 312264, 312260, 312257, 311310, 311308, 310761, 310760, 310628, 310609, 310605, 310604, 310603, 310602, 310600, 310599, 310598, 310596, 310595, 310593, 310590, 310589, 310586, 310583, 310569, 310567, 309741, 308604, 303321, 303311, 303310, 302727, 302723, 299593, 296702, 296252, 296251, 291928, 291922, 290026, 289827, 287437, 287432, 285674, 282325, 282324, 259283, 158724, 278299, 278298, 266200, 266197, 258245, 255509, 254329, 254325, 254322, 254319, 253139, 252677, 252675, 251876, 249407, 248854, 248328, 248326, 246081, 245599, 245494, 245109, 242989, 242698, 241967, 241966, 240625, 240623, 240618, 238654, 236574, 236206, 232626, 232625, 232638, 232637, 232636, 232635, 232631, 232630, 232629, 232628, 231937, 231935, 230268, 230107, 229583, 229581, 228492, 227658, 227657, 227069, 226745, 226188, 226171, 226073, 226072, 225605, 225601, 222770, 221940, 221939, 219395, 219394, 219146,

In [48]:
color_item = []
for index, href_items in enumerate(href):
# for index in range(1):
    s = Service(r"D:\CodeUIT\Data\Crawl-Data-Phone-Shop\chromedriver.exe")

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=s, options=chrome_options)

    driver.get(href_items)
    # sleep(random.randint(1,2))
    sleep(0.5)
    # Scroll down
    # for scroll_i in range(2):
    #     scroll_script = "window.scrollBy(0, window.innerHeight);"
    #     driver.execute_script(scroll_script)
    #     time.sleep(1) 
    print(id_product[index])

    # elems = driver.find_elements(By.CSS_SELECTOR, '.box03.color.group.desk a')
    # list_color = [elem.text for elem in elems if elem.text is not None]

    # len_color = len(elems)
    # len_loop = len_color+2
    
    elems_loop = driver.find_elements(By.CSS_SELECTOR, '.scrolling_inner .box01__tab.scrolling .item.itemTab')
    loop = [elem_loop for elem_loop in elems_loop]
    list_color = []

    for i in range(1,len(loop)):
        elem_find = driver.find_element(By.XPATH, "/html/body/section[1]/div[3]/div[1]/div[1]/div[2]/div/div[{}]".format(i))
        elem_find.click()

        elems = driver.find_elements(By.CSS_SELECTOR, '.item.itemTab.active')
        check = [elem.get_attribute('data-gallery-id') for elem in elems if elem.get_attribute('data-gallery-id') is not None]

        if check[0] != 'color-images-gallery':
            if check[0] == 'specification-gallery':
                break
            else: continue

        sleep(2)

        elems = driver.find_elements(By.CSS_SELECTOR, '.item.itemTab.active .item-border img')
        color = [elem.get_attribute('alt') for elem in elems if elem.get_attribute('alt') is not None]
        list_color.append(color[0])

        elems = driver.find_elements(By.CSS_SELECTOR, '.item.itemTab.active .item-border img')
        thumb_color = [elem.get_attribute('data-src') for elem in elems if elem.get_attribute('data-src') is not None]

        elems = driver.find_elements(By.CSS_SELECTOR, '.show-tab.active .owl-stage .owl-item img')
        href_color = [elem.get_attribute('data-src') for elem in elems if elem.get_attribute('data-src') is not None and 'jpg' in elem.get_attribute('data-src')]
        

        for item in href_color:
            if item is None or str(id_product[index]) not in item:
                href_color.remove(item)

        urls = href_color

        parent_folder = r'D:\CodeUIT\Data\imgs\imgs_details'
        folder_name = str(id_product[index])
        folder_path = os.path.join(parent_folder, folder_name, color[0])

        os.makedirs(folder_path, exist_ok=True)

        dowloadPath = folder_path
        
        thumb_color[0] = 'https:' + thumb_color[0]
        download_image_details(dowloadPath, thumb_color[0], 'thumb')

        for z, url in enumerate(urls):
            url = 'https:' + url
            download_image_details(dowloadPath, url, z)
    if len(list_color) >1 :
        color_item.append(', '.join(list_color[0:len(list_color)-1]))
    else: color_item.append(list_color[0])
    
    driver.close()

312263
Success  thumb
Success  0
Success  1
Success  2
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
311309
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
310592
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
309714
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
308532
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
307838
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  thumb
Succ

In [107]:
for index, href_items in enumerate(href_failed):
# for index in range(1):
    s = Service(r"D:\CodeUIT\Data\Crawl-Data-Phone-Shop\chromedriver.exe")

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=s, options=chrome_options)

    driver.get(href_items)
    # sleep(random.randint(1,2))
    sleep(0.5)
    # Scroll down
    # for scroll_i in range(2):
    #     scroll_script = "window.scrollBy(0, window.innerHeight);"
    #     driver.execute_script(scroll_script)
    #     time.sleep(1) 
    print(id_failed[index])

    elems = driver.find_elements(By.CSS_SELECTOR, '.box03.color.group.desk a')
    list_color = [elem.text for elem in elems if elem.text is not None]

    len_color = 1
    if len(elems) != 0:
        len_color = len(elems) 
    
    len_loop = len_color+2
    
    i = 1 
    while i < len_loop:
        elem_find = driver.find_element(By.XPATH, "/html/body/section[1]/div[3]/div[1]/div[1]/div[2]/div/div[{}]".format(i))
        elem_find.click()

        elems = driver.find_elements(By.CSS_SELECTOR, '.item.itemTab.active-scroll-to.active')
        check = [elem.get_attribute('data-gallery-id') for elem in elems if elem.get_attribute('data-gallery-id') is not None]

        if len(check) >0 :
            if check[0] != 'color-images-gallery' and check[0]:
                print(check)
                len_loop += 1
                i+=1
                continue
            else:
                i += 1
        else: break

        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.show-tab.active .owl-stage .owl-item img')))
        sleep(2)

        elems = driver.find_elements(By.CSS_SELECTOR, '.item.itemTab.active-scroll-to.active .item-border img')
        color = [elem.get_attribute('alt') for elem in elems if elem.get_attribute('alt') is not None]

        elems = driver.find_elements(By.CSS_SELECTOR, '.item.itemTab.active-scroll-to.active .item-border img')
        thumb_color = [elem.get_attribute('data-src') for elem in elems if elem.get_attribute('data-src') is not None]

        elems = driver.find_elements(By.CSS_SELECTOR, '.show-tab.active .owl-stage .owl-item img')
        href_color = [elem.get_attribute('data-src') for elem in elems if elem.get_attribute('data-src') is not None]
        

        for item in href_color:
            if item is None or str(id_failed[index]) not in item:
                href_color.remove(item)

        urls = href_color

        parent_folder = r'D:\CodeUIT\Data\imgs\imgs_details'
        folder_name = str(id_failed[index])
        folder_path = os.path.join(parent_folder, folder_name, color[0])

        os.makedirs(folder_path, exist_ok=True)

        dowloadPath = folder_path
        
        download_image_details(dowloadPath, thumb_color[0], 'thumb')

        for z, url in enumerate(urls):
            url = 'https:' + url
            download_image_details(dowloadPath, url, z)
    # color_item.append(', '.join(list_color))
    color_item[index_failed[index]] = color[0]
        

    driver.close()

305695
['featured-images-gallery']
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
Success  11
Success  12
Success  13
317163
Success  thumb
Success  0
Success  1
Success  2
Success  3
FAILED - cannot write mode P as JPEG
317164
Success  thumb
Success  0
Success  1
Success  2
Success  3
FAILED - cannot write mode P as JPEG
289694
['featured-images-gallery']
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
250622
['featured-images-gallery']
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
Success  11
Success  12
Success  13
285696
['featured-images-gallery']
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
Success  11
Success  12
Success  13
Success  14
Success  15
309821
['featured-images-gallery']
Succes

In [105]:
index_failed = []
for it in id_failed:
    for x, item in enumerate(id_product):
        if it == item:
            index_failed.append(x)

In [52]:
new_df2 = pd.DataFrame(list(zip(color_item)), columns = ['color_item'])
new_df2.to_excel('color_item_cable.xlsx', sheet_name='Sheet 1', index=False, engine='openpyxl')

In [53]:
color_item

['Đen, Trắng',
 'Trắng',
 'Trắng',
 'Xanh Navy',
 'Đen',
 'Xám',
 'Xanh Navy, Trắng',
 'Xanh Navy',
 'Trắng',
 'Đen',
 'Trắng, Xanh da trời nhạt',
 'Trắng',
 'Đen',
 'Đen',
 'Trắng, Đen',
 'Đen, Xanh Da Trời, Trắng, Xanh mint',
 'Đen',
 'Xám, Xanh Navy',
 'Đen, Bạc',
 'Đen, Bạc',
 'Đen',
 'Đen',
 'Trắng ngà/Xanh',
 'Trắng ngà/Xanh, Đen',
 'Trắng, Đen',
 'Xanh Dương',
 'Xanh Dương, Đen, Trắng',
 'Xanh Dương, Đen, Trắng',
 'Xanh Dương',
 'Trắng, Đen, Xanh Dương',
 'Tím, Hồng',
 'Trắng, Xanh da trời nhạt',
 'Trắng, Xanh da trời nhạt, Hồng, Tím',
 'Trắng',
 'Trắng, Đen',
 'Trắng, Đen',
 'Trắng, Đen',
 'Trắng, Đen',
 'Đen',
 'Xám',
 'Đen',
 'Xanh rêu',
 'Đen',
 'Trắng',
 'Xanh mint',
 'Đen',
 'Xanh Navy',
 'Xanh',
 'Xanh',
 'Trắng',
 'Xanh mint, Đen, Trắng',
 'Trắng',
 'Xanh',
 'Xanh Navy',
 'Xanh Navy',
 'Vàng',
 'Tím',
 'Tím',
 'Trắng',
 'Trắng',
 'Cam',
 'Xanh mint',
 'Xám',
 'Xanh ngọc',
 'Đen',
 'Xám',
 'Trắng',
 'Xanh Navy',
 'Xám',
 'Xám',
 'Hồng',
 'Đen',
 'Trắng, Đen',
 'Đen',
 'Xa